# Minority Rule data preparation
Joining Census Bureau demographic data with election data for future mapping.

TODO:
 - handle territories
 - clean up race category headers and include them

In [ ]:
import pandas as pd
import geopandas as gpd

## Counties
### County Demographics
Source: [2020 Census redistricting data](https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files.html)


In [ ]:
abbr_df = pd.read_csv('abbr.tsv', delimiter='\t', usecols=['State', 'Code'], index_col='Code')
abbr_lookup = abbr_df.to_dict()['State']
abbr_lookup

In [ ]:
# FIPS codes and county names to add FIPS code to demog data later on
# https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697

fips_df = pd.read_csv('county_fips.csv', dtype={'FIPS': str})
# todo: have the lookup handle territories - currently they'll have NaNs
fips_df['state_name'] = fips_df['State'].map(abbr_lookup)
fips_df['full_name'] = fips_df['Name'] + ', ' + fips_df['state_name']
fips_df = fips_df.set_index('full_name')
# fips_df.to_csv('county_fips.csv')
fips_df

In [ ]:
# census 2020 redistricting data
county_demog_df = pd.read_csv(
    'county-demog.csv',
    index_col='Label (Grouping)',
    skip_blank_lines=True,
    thousands=','
).transpose()
county_demog_df.index.name = 'county'
# remove county and parish to standardize across states
county_demog_df.index = county_demog_df.index.map(lambda x: x.replace(' County,', ',').replace(' Parish,', ','))
# combine demographic data with FIPS codes
county_demog_df = county_demog_df.join(fips_df).set_index('FIPS')
# limit demographics to only total pop for now
county_demog_df = county_demog_df[['Total:', 'Name', 'State', 'state_name']]
county_demog_df = county_demog_df.rename(
    {'Total:': 'total_pop', 'Name': 'name', 'State': 'state'}
)
county_demog_df

### County 2020 Presidential Election Results
Source: [County Presidential Election Returns 2000-2020](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ)

Dataset:
```
MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2020",
 https://doi.org/10.7910/DVN/VOQCHQ, Harvard Dataverse, V10,
 UNF:6:pVAMya52q7VM1Pl7EZMW0Q== [fileUNF]
```

File:
```
MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2020",
 https://doi.org/10.7910/DVN/VOQCHQ, Harvard Dataverse, V10;
 countypres_2000-2020.tab [fileName], UNF:6:pVAMya52q7VM1Pl7EZMW0Q== [fileUNF]
```

In [ ]:
county_pres_df = pd.read_csv(
    'countypres_2000-2020.tab',
    index_col=['county_fips'],
    delimiter='\t',
    dtype={'county_fips': str, 'version': str}
)

# limit to major parties for now
county_pres_df = county_pres_df[
    (county_pres_df['year'] == 2020) &
    (county_pres_df['party'].isin(('DEMOCRAT', 'REPUBLICAN')))
    ]

total_series = county_pres_df['totalvotes'].groupby(county_pres_df.index).max()

county_pres_df = county_pres_df.pivot_table(
    index=county_pres_df.index,
    columns='party',
    values='candidatevotes'
)
county_pres_df['TOTAL'] = total_series

county_pres_df['dem_percent'] = 100 * county_pres_df['DEMOCRAT'] / county_pres_df['TOTAL']
county_pres_df['rep_percent'] = 100 * county_pres_df['REPUBLICAN'] / county_pres_df['TOTAL']
county_pres_df['dem_lead'] = county_pres_df['DEMOCRAT'] - county_pres_df['REPUBLICAN']
county_pres_df['dem_percentage_pts_lead'] = county_pres_df['dem_percent'] - county_pres_df['rep_percent']

county_pres_df.to_csv('county_results.csv')
county_pres_df

### Combine to one table

In [ ]:
county_full_df = county_pres_df.join(county_demog_df)
county_full_df.index.name = 'fips'
county_full_df = county_full_df.rename(columns=str.lower)
county_full_df.to_csv('county-combined.csv')
county_full_df

## States
### State Demographics
Source: [2020 Census redistricting data](https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files.html)



In [ ]:
state_demog_df = pd.read_csv(
    'state-demog.csv',
    index_col='Label (Grouping)',
    skip_blank_lines=True,
    thousands=','
).transpose()

state_demog_df.index.name = 'state'
state_demog_df.index = state_demog_df.index.str.upper()

# limit demographics to only total pop for now
state_demog_df = state_demog_df[['Total:']]
state_demog_df = state_demog_df.rename(columns={'Total:': 'total_pop'})

state_demog_df

## State 2020 Presidential Election Results
Source: [U.S. President 1976–2020](https://dataverse.harvard.edu/file.xhtml?fileId=4299753&version=6.0)

Dataset
```
 MIT Election Data and Science Lab, 2017, "U.S. President 1976–2020",
  https://doi.org/10.7910/DVN/42MVDX, Harvard Dataverse, V6,
  UNF:6:4KoNz9KgTkXy0ZBxJ9ZkOw== [fileUNF]
```

File
```
 MIT Election Data and Science Lab, 2017, "U.S. President 1976–2020",
  https://doi.org/10.7910/DVN/42MVDX, Harvard Dataverse, V6;
  1976-2020-president.tab [fileName], UNF:6:4KoNz9KgTkXy0ZBxJ9ZkOw== [fileUNF]
```


In [ ]:
state_elect_df = pd.read_csv('1976-2020-president.tab', delimiter='\t', index_col='state')
state_elect_df = state_elect_df[
    (state_elect_df['year'] == 2020) & (state_elect_df['party_simplified'].isin(['DEMOCRAT', 'REPUBLICAN']))]

total_series = state_elect_df['totalvotes'].groupby(state_elect_df.index).max()

state_elect_df = state_elect_df.pivot_table(index=state_elect_df.index, columns='party_simplified',
                                            values='candidatevotes')
state_elect_df['TOTAL'] = total_series
state_elect_df['dem_percent'] = 100 * state_elect_df['DEMOCRAT'] / state_elect_df['TOTAL']
state_elect_df['rep_percent'] = 100 * state_elect_df['REPUBLICAN'] / state_elect_df['TOTAL']
state_elect_df['dem_lead'] = state_elect_df['DEMOCRAT'] - state_elect_df['REPUBLICAN']
state_elect_df['dem_percentage_pts_lead'] = state_elect_df['dem_percent'] - state_elect_df['rep_percent']
state_elect_df.to_csv('state_results.csv')
state_elect_df

### Combine to one table

In [ ]:
state_full_df = state_elect_df.join(state_demog_df)
state_full_df.index.name = 'state'
state_full_df = state_full_df.rename(columns=str.lower)
state_full_df.to_csv('state-combined.csv')
state_full_df

## Create GeoJSONs
Source: [US Census Bureau Cartographic Boundary Files](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)

In [ ]:
county_geo_df = gpd.read_file('cb_2018_us_county_20m/cb_2018_us_county_20m.shp')
county_geo_df['NAME'] = county_geo_df['NAME'].str.upper()
county_geo_df = county_geo_df.join(county_full_df, on='GEOID')
county_geo_df = county_geo_df.set_index('GEOID')
county_geo_df.to_file('counties.geojson', driver='GeoJSON')
county_geo_df

In [ ]:
state_geo_df = gpd.read_file('cb_2018_us_state_20m/cb_2018_us_state_20m.shp')
state_geo_df['NAME'] = state_geo_df['NAME'].str.upper()
state_geo_df = state_geo_df.join(state_full_df, on='NAME')
state_geo_df = state_geo_df.set_index('STATEFP')
state_geo_df.to_file('states.geojson', driver='GeoJSON')
state_geo_df